In [3]:
import pandas as pd
import numpy as np
import scipy.stats as st
import plotly.express as px
import plotly.graph_objects as go
import psycopg2
import random
from plotly.subplots import make_subplots
from decouple import config


In [4]:
def connection(sql):
    con = psycopg2.connect(host='35.247.202.234', database='dw',
    user='consulta', password='teste123')
    cur = con.cursor()    
    cur.execute(sql)
    data = cur.fetchall()
    cols = []
    for elt in cur.description:
        cols.append(elt[0])

    df = pd.DataFrame(data=data,columns=cols)
    con.close();
    return df

In [56]:
def velocimetro_dimensao():
    # dataframe para os indices fatores medio baixo e medio alto
    d = {'Dimensao': ['E-ESC', 'E-PROF', 'E-FAM', 'E-COM', 'E-EST'],
         'Medio_Baixo': [2.84, 2.67, 3.34, 2.45, 2.44],
         'Medio_Alto': [4, 3.66, 4.33, 3.55, 3.41]}
    dimensoes = pd.DataFrame(data=d)
    #print(dimensoes)
    query = """select * from public.dimensoes_est de
    inner join escolas.aluno a 
    on de.id_aluno = a.id_aluno 
    inner join escolas.turma t 
    on a.id_turma = t.id_turma 
    inner join escolas.escola e 
    on t.id_escola = e.cod_escola"""
    # df = pd.read_sql(con_db_caio2(query))
    df = connection(query)


    df = df.loc[:, ~df.columns.duplicated()].copy()

    cross_tab_prop = df[
        ['id_aluno', 'nome_aluno', 'id_turma', 'escola', 'uf', 'municipio', 'cod_escola', 'nome_turma', 
         'E_ESCV','E_PROFV','E_FAMV', 'E_COMV','E_ESTV',
         'E_ESCC',  'E_PROFC', 'E_FAMC', 'E_COMC',  'E_ESTC',]]

    cross_tab_prop.set_index('id_aluno')
    # cross_tab_prop

  
    nomeEscola = 'E M E F PROFESSORA DALILA LEAO'
    filter = cross_tab_prop['escola']==nomeEscola
    escola_dimensao = cross_tab_prop[filter]
 

    #filter = (cross_tab_prop['escola']=='E M E F PROFESSORA DALILA LEAO') & (cross_tab_prop['id_turma']==1064)
    #turma_dimensao = cross_tab_prop[filter]
  
    
    #filter = (cross_tab_prop['escola']=='E M E F PROFESSORA DALILA LEAO') & (cross_tab_prop['id_turma']==1064) & (cross_tab_prop['nome_aluno']=='Jeferson Oliveira dos Santos')
    #aluno_dimensao = cross_tab_prop[filter]
    dictionary_name = { 'E-ESC':   'Dimensão Estudante-Escola',
                        'E-PROF': 'Dimensão Estudante-Profissionais',
                        'E-FAM' : 'Dimensão Estudante-Familia',
                        'E-COM' : 'Dimensão Estudante-Comunidade',
                        'E-EST' : 'Dimensão Estudante-Estudante'}


    cols = escola_dimensao.loc[:, escola_dimensao.columns.str.startswith("E_")]
    cols = cols.loc[:, cols.columns.str.endswith("V")]
    r = 1
    c = 1
    n = 1
    tracer = {}
    
    for (columnName, columnData) in cols.iteritems():
        cols[columnName] = cols[columnName].astype(np.float16)
        mean = cols[columnName].mean()
       
        nameC = columnName
        nameC = nameC.replace('V', "")
        f2 = dimensoes[dimensoes['Dimensao'] == nameC.replace('_', "-")]
        columnName =nameC.replace('_', "-")
        tracer[n] = go.Indicator(
            mode="gauge+number+delta",
            value=mean,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': dictionary_name[columnName], 'font': {'size': 18}},
            # reference é pode ser a média geral
            delta={'reference': f2['Medio_Baixo'].values[0], 'increasing': {'color': "RebeccaPurple"}},
            gauge={
                'axis': {'range': [None, 7], 'tickwidth': 1, 'tickcolor': "darkblue"},
                'bar': {'color': "darkblue"},
                'bgcolor': "red",
                'borderwidth': 2,
                'bordercolor': "gray",
                'steps': [
                    {'range': [0, f2['Medio_Baixo'].values[0]], 'color': 'green'},
                    {'range': [f2['Medio_Baixo'].values[0], f2['Medio_Alto'].values[0]], 'color': 'yellow'},
                    {'range': [f2['Medio_Alto'].values[0], 7], 'color': 'red'}

                ],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 490}})
        r = 1
        c = 1
        n = n + 1

    fig = make_subplots(
        rows=4,
        cols=3,
        specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
               [{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
               [{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
               [{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],

               ],
    )
    for i in range(1, 6):
        if (c % 2 != 0):
            # fig.append_trace(trace1, row=1, col=1)
            # fig.append_trace(trace, row=r, col=c)
            # r = r + 1
            # print ('if, r: ', r, ' C: ', c)
            fig.append_trace(tracer[i], row=r, col=c)
            c = c + 1

        else:
            # print ('else, r: ', r, ' C: ', c)
            fig.append_trace(tracer[i], row=r, col=c)
            c = 1
            r = r + 1

            # fig.append_trace(trace, row=r, col=c)

    fig.update_layout(height=1000, width=1200,title_y=0.98,title_x=0.25, title_text="<b>Dimensoes da {0}</b>".format(nomeEscola))
    # fig.show()
    return fig

In [57]:
fig = velocimetro_dimensao()
fig.show()

/tmp/ipykernel_11826/106618741.py:55: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [65]:
def velocimetro_fator():
    #dataframe para os indices fatores medio baixo e medio alto
    d = {'Fator': ['E_ESC1', 'E_ESC2', 'E_PROF1', 'E_PROF2', 'E_FAM1', 'E_FAM2', 'E_COM1', 'E_COM2', 'E_COM3', 'E_EST1',
               'E_EST2', 'E_EST3'], 'Medio_Baixo': [3.66, 2.01, 2.34, 2.34, 3.01, 3.34, 2.01, 2.01, 2.01, 2.01, 2.34, 2.01], 
     'Medio_Alto': [5.33, 3.33, 4.33, 4.00, 5.00, 5.00, 3.66, 3.66, 3.66, 4.00, 4.00, 3.33]}
    fatores = pd.DataFrame(data = d)
    query = """select * from public.fatores_est fe
    inner join escolas.aluno a 
    on fe.id_aluno = a.id_aluno 
    inner join escolas.turma t 
    on a.id_turma = t.id_turma 
    inner join escolas.escola e 
    on t.id_escola = e.cod_escola"""
    #df = pd.read_sql(con_db_caio2(query))
    df = connection(query)

    df = df.loc[:,~df.columns.duplicated()].copy()
    
    cross_tab_prop = df[['id_aluno','nome_aluno','id_turma', 'escola', 'uf', 'municipio','cod_escola','nome_turma','E_ESC1V', 'E_ESC2V', 'E_PROF1V', 'E_PROF2V',
       'E_FAM1V', 'E_FAM2V', 'E_COM1V', 'E_COM2V', 'E_COM3V', 'E_EST1V',
       'E_EST2V', 'E_EST3V', 'E_ESC1C', 'E_ESC2C', 'E_PROF1C', 'E_PROF2C',
       'E_FAM1C', 'E_FAM2C', 'E_COM1C', 'E_COM2C', 'E_COM3C', 'E_EST1C',
       'E_EST2C', 'E_EST3C']]

    cross_tab_prop.set_index('id_aluno')
    #cross_tab_prop
    
    filter_column = cross_tab_prop.columns.str.startswith("E_") | cross_tab_prop.columns.str.startswith("id") | cross_tab_prop.columns.str.startswith("cod")|cross_tab_prop.columns.str.startswith("nome")|cross_tab_prop.columns.str.startswith("escola") |cross_tab_prop.columns.str.startswith("uf")|cross_tab_prop.columns.str.startswith("municipio")
    df_col = cross_tab_prop.loc[:,filter_column]
    filter_column = df_col.columns.str.endswith("V") | df_col.columns.str.startswith("id") | df_col.columns.str.startswith("cod")|df_col.columns.str.startswith("nome")|df_col.columns.str.startswith("escola") |df_col.columns.str.startswith("uf")|df_col.columns.str.startswith("municipio")
    df_col = df_col.loc[:, filter_column]
    #df_col.info()
    
    df_col['E_ESCV'] = (df_col['E_ESC2V'] +df_col['E_ESC2V'])/2
    df_col['E_PROFV'] = (df_col['E_PROF1V'] +df_col['E_PROF1V'])/2
    df_col['E_FAMV'] = (df_col['E_FAM1V'] +df_col['E_FAM1V'])/2
    df_col['E_COMV'] = (df_col['E_COM1V'] +df_col['E_COM2V']+df_col['E_COM3V'])/3
    df_col['E_ESTV'] = (df_col['E_EST1V'] +df_col['E_EST2V']+df_col['E_EST3V'])/3
    
    nomeEscola = 'E M E F PROFESSORA DALILA LEAO'
    filter = cross_tab_prop['escola']==nomeEscola
    escola_fator = cross_tab_prop[filter]
    
    
    filter = (cross_tab_prop['escola']=='E M E F PROFESSORA DALILA LEAO') & (cross_tab_prop['id_turma']==1064)
    turma_fator = cross_tab_prop[filter]

    
    filter = (cross_tab_prop['escola']=='E M E F PROFESSORA DALILA LEAO') & (cross_tab_prop['id_turma']==1064) & (cross_tab_prop['nome_aluno']=='Jeferson Oliveira dos Santos')
    aluno_fator = cross_tab_prop[filter]

    
    cols = escola_fator.loc[:, escola_fator.columns.str.startswith("E_")]
    cols = cols.loc[:, cols.columns.str.endswith("V")]
    r = 1
    c = 1
    n = 1
    tracer = {}
    dictionary_name = { 'E-ESC':   'Dimensão Estudante-Escola',
                        'E-PROF': 'Dimensão Estudante-Profissionais',
                        'E-FAM' : 'Dimensão Estudante-Familia',
                        'E-COM' : 'Dimensão Estudante-Comunidade',
                        'E-EST' : 'Dimensão Estudante-Estudante'}

    for (columnName, columnData) in cols.iteritems():
        
        cols[columnName] = cols[columnName].astype(np.float16)
        mean = cols[columnName].mean()
        
        nameC = columnName
        nameC = nameC.replace('V',"")
        f2 = fatores[fatores['Fator'] == nameC]
        
        
        tracer[n] = go.Indicator(
            mode = "gauge+number+delta",
            value = mean,
            domain = {'x': [0, 1], 'y': [0, 1]},
            title = {'text': columnName, 'font': {'size': 24}},
            #reference é pode ser a média geral
            delta = {'reference': f2['Medio_Baixo'].values[0], 'increasing': {'color': "RebeccaPurple"}},
            gauge = {
                'axis': {'range': [None, 7], 'tickwidth': 1, 'tickcolor': "darkblue"},
                'bar': {'color': "darkblue"},
                'bgcolor': "red",
                'borderwidth': 2,
                'bordercolor': "gray",
                'steps': [
                    {'range': [0, f2['Medio_Baixo'].values[0]], 'color': 'green'},
                    {'range': [f2['Medio_Baixo'].values[0], f2['Medio_Alto'].values[0]], 'color': 'yellow'},
                    {'range': [f2['Medio_Alto'].values[0], 7], 'color': 'red'}
                    
                ],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 490}})
        r = 1
        c = 1
        n = n + 1



    fig = make_subplots(
        rows=4,
        cols=3,
        specs=[[{'type' : 'indicator'}, {'type' : 'indicator'}, {'type' : 'indicator'}],
              [{'type' : 'indicator'}, {'type' : 'indicator'}, {'type' : 'indicator'}],
               [{'type' : 'indicator'}, {'type' : 'indicator'}, {'type' : 'indicator'}],
               [{'type' : 'indicator'}, {'type' : 'indicator'}, {'type' : 'indicator'}],

        ],
        )
    for i in range (1, 13):
            if(c%3 != 0):
                #fig.append_trace(trace1, row=1, col=1)
                #fig.append_trace(trace, row=r, col=c)
                #r = r + 1
                #print ('if, r: ', r, ' C: ', c)
                fig.append_trace(tracer[i], row=r, col=c)
                c = c + 1

            else:
                #print ('else, r: ', r, ' C: ', c)
                fig.append_trace(tracer[i], row=r, col=c)
                c = 1
                r = r + 1

                #fig.append_trace(trace, row=r, col=c)

    fig.update_layout(height=1000, width=1200,title_y=0.99,title_x=0.25, title_text="<b>Fatores da {0}</b>".format(nomeEscola))
    #fig.show()
    return fig

In [66]:
fig = velocimetro_fator()
fig.show()

/tmp/ipykernel_11826/2986451656.py:60: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

